<a href="https://colab.research.google.com/github/Rahulm94/PySpark-Streaming-Example/blob/main/PySpark_Streaming_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

from google.colab import files
uploaded = files.upload()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0479af9129c5b1a653444357fa0208ac1f332231b95228103648bdae094d38f1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Saving heart.csv to heart.csv


In [2]:
import os

print(os.getcwd())

/content


In [3]:
INPUT_PATH = '/content/input'
OUTPUT_PATH = '/content/output'

if not(os.path.exists(INPUT_PATH)):
  os.mkdir(INPUT_PATH)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,LongType, StringType,DoubleType,TimestampType
from pyspark.sql.functions import round

spark = SparkSession.builder.appName('Spark Streaming').getOrCreate()

In [5]:
# Define the schema

schema = StructType([StructField("age", LongType(), True), StructField("sex", LongType(), True),
                    StructField("cp", LongType(), True), StructField('trtbps', LongType(), True),
                    StructField("chol", LongType(), True), StructField("fbs", LongType(), True),
                    StructField("restecg", LongType(), True), StructField("thalachh", LongType(), True),
                    StructField("exng", LongType(), True), StructField("oldpeak", DoubleType(), True),
                    StructField("slp", LongType(),True), StructField("caa", LongType(), True),
                    StructField("thall", LongType(), True), StructField("output", LongType(), True)])

data = "heart.csv"
df = spark.read.format('csv').option('header', True).schema(schema).load(data)
df.withColumnRenamed("output", "label").show()

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+-----+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|label|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+-----+
| 63|  1|  3|   145| 233|  1|      0|     150|   0|    2.3|  0|  0|    1|    1|
| 37|  1|  2|   130| 250|  0|      1|     187|   0|    3.5|  0|  0|    2|    1|
| 41|  0|  1|   130| 204|  0|      0|     172|   0|    1.4|  2|  0|    2|    1|
| 56|  1|  1|   120| 236|  0|      1|     178|   0|    0.8|  2|  0|    2|    1|
| 57|  0|  0|   120| 354|  0|      1|     163|   1|    0.6|  2|  0|    2|    1|
| 57|  1|  0|   140| 192|  0|      1|     148|   0|    0.4|  1|  0|    1|    1|
| 56|  0|  1|   140| 294|  0|      0|     153|   0|    1.3|  1|  0|    2|    1|
| 44|  1|  1|   120| 263|  0|      1|     173|   0|    0.0|  2|  0|    3|    1|
| 52|  1|  2|   172| 199|  1|      1|     162|   0|    0.5|  2|  0|    3|    1|
| 57|  1|  2|   150| 168|  0|      1|   

In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- sex: long (nullable = true)
 |-- cp: long (nullable = true)
 |-- trtbps: long (nullable = true)
 |-- chol: long (nullable = true)
 |-- fbs: long (nullable = true)
 |-- restecg: long (nullable = true)
 |-- thalachh: long (nullable = true)
 |-- exng: long (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slp: long (nullable = true)
 |-- caa: long (nullable = true)
 |-- thall: long (nullable = true)
 |-- output: long (nullable = true)



In [7]:
# Train test

trainDF, testDF = df.randomSplit([0.7, 0.3])

In [8]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.functions import vector_to_array
from pyspark.ml import Pipeline

In [9]:
# Min max scaler
inputs = ['age','trtbps','chol','thalachh','oldpeak']
assembler1 = VectorAssembler(inputCols=inputs, outputCol="features_to_scale")
scaler = MinMaxScaler(inputCol="features_to_scale", outputCol="features_scaled")

# One hot encoder
ohe = OneHotEncoder(inputCols = ['sex', 'cp', 'fbs', 'restecg', 'slp', 'exng', 'caa', 'thall'],
                    outputCols=['sex_ohe', 'cp_ohe', 'fbs_ohe', 'restecg_ohe', 'slp_ohe', 'exng_ohe', 'caa_ohe', 'thall_ohe'],
                    handleInvalid="keep")
assembler2 = VectorAssembler(inputCols=['sex_ohe', 'cp_ohe', 'fbs_ohe', 'restecg_ohe', 'slp_ohe', 'exng_ohe', 'caa_ohe', 'thall_ohe','features_scaled'],
                              outputCol="features",
                              handleInvalid="keep")

# Create logistic regression model
lr = LogisticRegression(maxIter=50, regParam=0.1)

# Create pipeline

myStages = [assembler1, scaler, ohe, assembler2, lr]
pipeline = Pipeline(stages=myStages)

In [10]:
# Train the model

trainDF = trainDF.withColumnRenamed("output", "label")
lr_model = pipeline.fit(trainDF)

# Transform the data to predictions

trainingPred = lr_model.transform(trainDF)
trainingPred.select('label','probability','prediction').show()

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|    1|[0.07477051882632...|       1.0|
|    1|[0.02459259610929...|       1.0|
|    1|[0.07760452733787...|       1.0|
|    1|[0.06444690566470...|       1.0|
|    0|[0.68612043046326...|       0.0|
|    1|[0.06432959538877...|       1.0|
|    1|[0.02300469105943...|       1.0|
|    1|[0.15154553553101...|       1.0|
|    0|[0.60690521710507...|       0.0|
|    1|[0.01894396772976...|       1.0|
|    1|[0.04978942563959...|       1.0|
|    0|[0.64604355188096...|       0.0|
|    1|[0.12809459077549...|       1.0|
|    0|[0.86585543380385...|       0.0|
|    0|[0.35334806785956...|       1.0|
|    1|[0.06905181458773...|       1.0|
|    1|[0.04282710232077...|       1.0|
|    1|[0.05904225920921...|       1.0|
|    1|[0.07069272876737...|       1.0|
|    0|[0.45631752358433...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



In [11]:
df_pred = trainingPred.select('label','probability','prediction').toPandas()
df_pred.head()

,label,probability,prediction
0,1,"[0.07477051882632169, 0.9252294811736783]",1.0
1,1,"[0.02459259610929501, 0.975407403890705]",1.0
2,1,"[0.0776045273378737, 0.9223954726621263]",1.0
3,1,"[0.06444690566470784, 0.9355530943352921]",1.0
4,0,"[0.6861204304632644, 0.31387956953673557]",0.0


In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction',metricName='f1')
score = evaluator.evaluate(trainingPred)
print(score)

0.8597254811747566


In [13]:
testDF = testDF.drop('output').toPandas()
for i in range(0,testDF.shape[0]//10+1):
  path = f"{INPUT_PATH}/heartTest{i}.csv"
  if not(os.path.exists(path)):
    testDF.iloc[i*10:(i+1)*10,:].to_csv(path, index=False)

In [14]:
# Incorporate streaming source data
sourceStream = spark.readStream.format("csv").option("header",True).schema(schema[:-1]).option('enforceSchema','false') \
                               .option("ignoreLeadingWhiteSpace",True).option("mode","PERMISSIVE") \
                               .option("maxFilesPerTrigger",1).load(f"{INPUT_PATH}")\


streamingHeart = lr_model.transform(sourceStream).select('prediction',vector_to_array('probability'))\
                                              .withColumnRenamed("UDF(probability)", "probability")
streamingHeart = streamingHeart.withColumn('probability_0', streamingHeart["probability"].getItem(0))\
                               .withColumn('probability_1', streamingHeart["probability"].getItem(1))\
                               .drop('probability').select('prediction', round('probability_0',4).alias('probability_0')
                               ,round('probability_1',4).alias('probability_1'))

query = streamingHeart.writeStream.format("csv").option("header","true") \
                                  .option("checkpointLocation", "checkpoint/") \
                                  .option("path", f"{OUTPUT_PATH}") \
                                  .outputMode("append").start()
query.awaitTermination()